In [37]:
import math
import pandas as pd
import numpy as np
import random
from deap import creator, base, tools, algorithms
from sklearn import svm
from sklearn.preprocessing import StandardScaler
def avg(l):
    """
    Returns the average between list elements
    """
    return (sum(l)/float(len(l)))

In [38]:
def calculate_equivalence_classes(data, condition_attrs):
    # 取出条件属性的列索引
    # 设置条件属性为索引
    # print(condition_attrs)
    data_indexed = data.set_index(condition_attrs)
    # 按照索引分组并将分组内结果合并为列表
    equivalence_classes = data_indexed.groupby(level=condition_attrs)[data.columns[0]].apply(list).to_dict()
    return equivalence_classes

def getFitness(data,decision_attr,individual):
    # 计算D的信息熵
    total_records = len(data)
    decision_counts = data[decision_attr].value_counts()
    decision_prob = decision_counts / total_records
    entropy_D = -(decision_prob * np.log2(decision_prob)).sum()

    selected_features = [feature_columns[i] for i in range(len(feature_columns)) if individual[i] == 1]

    equivalence_classes = calculate_equivalence_classes(data,selected_features)
    equivalence_classes_dec = calculate_equivalence_classes(data, decision_attr)

    rule_confidences = []
    entropy_DB = 0
    total_rule=0

    for equivalence_class in equivalence_classes.values():
        class_size = len(equivalence_class)
        class_prob = class_size / total_records
        for key_decision in equivalence_classes_dec.keys():
            x=set(equivalence_class).intersection(equivalence_classes_dec.get(key_decision))
            if(len(x)>total_records*0):
                total_rule+=len(x)
                y = 1-len(x) / class_size
                rule_confidences.append(y)
            if (len(x)>0):
                decision_prob = len(x) / class_size
                entropy_DB -= class_prob * decision_prob * math.log(decision_prob, 2)
    rule_confidences = np.array(rule_confidences)
    count = len(rule_confidences)
    if count == 0:
        return(999,);
#    if total_rule/total_records<0.3:
#        return (999,);

    avg_confidence = np.sum(rule_confidences) / count +wight1*(entropy_D-entropy_DB)+wight2*(count/total_records)
    return(avg_confidence,)

def geneticAlgorithm(df,y,n_population, n_generation):
    """
    Deap global variables
    Initialize variables to use eaSimple
    """
    # create individual
    random.seed(42)
    # random.seed(1)
    creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMin)

    # create toolbox
    toolbox = base.Toolbox()
    toolbox.register("attr_bool", random.randint, 0, 1)
    toolbox.register("individual", tools.initRepeat,
                     creator.Individual, toolbox.attr_bool, len(X.columns))
    toolbox.register("population", tools.initRepeat, list,
                     toolbox.individual)
    toolbox.register("evaluate", getFitness,df,y)
    toolbox.register("mate", tools.cxOnePoint)
    toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
    toolbox.register("select", tools.selTournament, tournsize=3)

    # initialize parameters
    pop = toolbox.population(n=n_population)
    hof = tools.HallOfFame(n_population * n_generation)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("min", np.min)
    # stats.register("max", np.max)

    # genetic algorithm
    pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2,
                                   ngen=n_generation, stats=stats, halloffame=hof,
                                   verbose=True)

    # return hall of fame
    top_5_hof = sorted(hof.items, key=lambda x: x.fitness.values[0])[:1]
    for i, ind in enumerate(top_5_hof):
        print(f"Top {i+1} solution: {ind}\nFitness: {ind.fitness.values[0]}\n")
    return top_5_hof

In [39]:
from sklearn.neighbors import KNeighborsClassifier

def bestIndividual_KNN(hof,df):
    k = 6
    knn = KNeighborsClassifier(n_neighbors=k)
    for ind in hof:
        df = pd.read_csv('data/segment/segmentall.csv')
        df.drop('REGION-PIXEL-COUNT', axis=1, inplace=True)
        feature_columns = [col for col in df.columns if col not in ['id','target']]
        selected_features = [feature_columns[i] for i in range(len(feature_columns)) if ind[i] == 1]
        Y=df["target"]
        X=df[selected_features]
        kf = KFold(n_splits=10, shuffle=True, random_state=42)
        accuracy_scores = []
        for train_index, test_index in kf.split(X):
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = Y.iloc[train_index], Y.iloc[test_index]

            knn.fit(X_train, y_train)
            y_pred = knn.predict(X_test)
            accuracy = accuracy_score(y_test, y_pred)
            accuracy_scores.append(accuracy)
        mean_accuracy = sum(accuracy_scores) / len(accuracy_scores)
        return mean_accuracy,selected_features

In [40]:
from sklearn.model_selection import StratifiedKFold

def bestIndividual_SVM(hof):
    for ind in hof:
        df = pd.read_csv('data/segment/segmentall.csv')
        df.drop('REGION-PIXEL-COUNT', axis=1, inplace=True)
        feature_columns = [col for col in df.columns if col not in ['id', 'target']]
        selected_features = [feature_columns[i] for i in range(len(feature_columns)) if ind[i] == 1]
        Y = df["target"]
        X = df[selected_features]

        # Use stratified k-fold cross-validation
        skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
        scores = []
        for train_index, test_index in skf.split(X, Y):
            trainX, testX = X.iloc[train_index], X.iloc[test_index]
            trainY, testY = Y.iloc[train_index], Y.iloc[test_index]
            score = SVM(trainX, trainY, testX, testY)
            scores.append(score)
        avg_score = np.mean(scores)

        if avg_score > maxaccuracy:
            maxaccuracy = avg_score

    return maxaccuracy

In [41]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier


def bestIndividual_DT(hof):
    for ind in hof:
        df = pd.read_csv('data/segment/segmentall.csv')
        df.drop('REGION-PIXEL-COUNT', axis=1, inplace=True)
        feature_columns = [col for col in df.columns if col not in ['id','target']]
        selected_features = [feature_columns[i] for i in range(len(feature_columns)) if ind[i] == 1]
        X = df[selected_features]
        Y = df['target']
        dt = DecisionTreeClassifier(max_depth=7)
        kf = KFold(n_splits=10, shuffle=True, random_state=42)

        accuracy_scores = []
        for train_index, test_index in kf.split(X):
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = Y.iloc[train_index], Y.iloc[test_index]

            dt.fit(X_train, y_train)

            y_pred = dt.predict(X_test)
            accuracy = accuracy_score(y_test, y_pred)
            accuracy_scores.append(accuracy)

        mean_accuracy = sum(accuracy_scores) / len(accuracy_scores)
    return mean_accuracy

In [42]:
wighta=[0,0.1,0.2,0.3,0.4,0.5]
wightb=[0,0.01,0.02,0.03,0.04,0.05]
SVM=[[0 for _ in range(len(wighta))] for _ in range(len(wighta))]
KNN=[[0 for _ in range(len(wighta))] for _ in range(len(wighta))]
DT=[[0 for _ in range(len(wighta))] for _ in range(len(wighta))]
F=[[0 for _ in range(len(wighta))] for _ in range(len(wighta))]


for i_index, i in enumerate(wighta):
    for j_index, j in enumerate(wightb):
        try:
            # 你的代码
            wight1=i
            wight2=j
            n_pop=50
            n_gen=30
            # read dataframe from csv
            df = pd.read_csv('data/segment/segmentall.csv')
            df.drop('REGION-PIXEL-COUNT', axis=1, inplace=True)
            # encode labels column to numbers
            y = df['target']
            feature_columns = [ col for col in df.columns if col not in ['id','target']]
            X = df[feature_columns]
            feature_columns = [ col for col in df.columns if col not in ['id','target','SHORT-LINE-DENSITY-5','SHORT-LINE-DENSITY-2']]
            # feature_columns=['f4','f5','f6','f7','f17','f18']
            for col in feature_columns:
                df[col] = pd.qcut(df[col], q=3, labels=False, duplicates='drop')
            # get accuracy with all features
            individual = [1 for i in range(len(X.columns))]
            getFitness(df,'target',individual)
            print("Accuracy with all features: \t" +
                  str(getFitness(df,'target',individual)) + "\n")

            # apply genetic algorithm
            hof = geneticAlgorithm(df,'target',n_pop, n_gen)

            print('------------wight1--------------')
            print(wight1)
            print('------------wight2--------------')
            print(wight2)

            accuracy_SVM,individual = bestIndividual_SVM(hof,df)
            SVM[i_index][j_index] = accuracy_SVM
            print("-----------SVM---------------")
            print(accuracy_SVM, "保存至 SVM[{}][{}]".format(i_index, j_index))

            accuracy_KNN,individual = bestIndividual_KNN(hof,df)
            KNN[i_index][j_index] = accuracy_KNN
            print("-----------KNN---------------")
            print(accuracy_KNN, "保存至 KNN[{}][{}]".format(i_index, j_index))

            accuracy_DT = bestIndividual_DT(hof,df)
            DT[i_index][j_index] = accuracy_DT
            print("-----------DT---------------")
            print(accuracy_DT, "保存至 DT[{}][{}]".format(i_index, j_index))

            print('------------特征-----------------')
            print(individual)
            F[i_index][j_index] = individual
            print("特征保存至 F[{}][{}]".format(i_index, j_index))

        except Exception as e:
            print("执行过程出现错误：", e)
            continue

Accuracy with all features: 	(0.014073494917904612,)



C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


gen	nevals	avg     	min      
0  	50    	0.218618	0.0357143
1  	27    	0.127667	0.0215558
2  	31    	0.0730904	0.0209281
3  	37    	0.0444383	0.0197842
4  	34    	0.0284971	0.0197842
5  	31    	0.0231539	0.0150316
6  	31    	0.0215915	0.0148392
7  	34    	0.0205082	0.0141398
8  	35    	0.0200884	0.0140735
9  	30    	0.0179158	0.0140735
10 	21    	0.0151261	0.0140735
11 	22    	0.0144035	0.0140735
12 	30    	0.0143889	0.0140735
13 	37    	0.0145896	0.0140735
14 	31    	0.0154952	0.0140735
15 	29    	0.0143745	0.0140735
16 	33    	0.0171069	0.0140735
17 	27    	0.0167386	0.0140735
18 	32    	0.0157471	0.0140735
19 	32    	0.0145187	0.0140735
20 	32    	0.0163258	0.0140735
21 	35    	0.01589  	0.0140735
22 	31    	0.0143657	0.0140735
23 	26    	0.0154372	0.0140735
24 	35    	0.0149812	0.0140735
25 	23    	0.0153039	0.0140735
26 	29    	0.0147635	0.0140735
27 	31    	0.01518  	0.0140735
28 	33    	0.0160664	0.0140735
29 	25    	0.0143113	0.0140735
30 	30    	0.015054 	0.0140735
Top 1 solut

C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


gen	nevals	avg     	min      
0  	50    	0.220863	0.0392294
1  	27    	0.130345	0.0261748
2  	31    	0.0763032	0.0256857
3  	37    	0.048211 	0.024598 
4  	34    	0.0328181	0.024598 
5  	31    	0.0277091	0.0205035
6  	31    	0.0262893	0.0200903
7  	34    	0.0254086	0.0196506
8  	35    	0.0250114	0.0196103
9  	30    	0.0230785	0.0196103
10 	21    	0.0205142	0.0196103
11 	22    	0.0198781	0.0196103
12 	30    	0.0198976	0.0196103
13 	37    	0.0200995	0.0196103
14 	31    	0.0209844	0.0196103
15 	29    	0.0198887	0.0196103
16 	33    	0.0225334	0.0196103
17 	27    	0.0221422	0.0196103
18 	32    	0.0211789	0.0196103
19 	32    	0.0200038	0.0196103
20 	32    	0.0217414	0.0196103
21 	35    	0.0213456	0.0196103
22 	31    	0.0198584	0.0196103
23 	26    	0.0208938	0.0196103
24 	35    	0.0204495	0.0196103
25 	23    	0.0207948	0.0196103
26 	29    	0.0202792	0.0196103
27 	31    	0.0206638	0.0196103
28 	33    	0.0215416	0.0196103
29 	25    	0.0198297	0.0196103
30 	30    	0.0205394	0.0196103
Top 1 solut

...

gen	nevals	avg    	min    
0  	50    	1.42391	1.30325
1  	27    	1.39634	1.23982
2  	31    	1.37687	1.28468
3  	37    	1.36167	1.26635
4  	34    	1.34755	1.26635
5  	31    	1.32432	1.18931
6  	31    	1.28151	1.18931
7  	34    	1.25441	1.18931
8  	35    	1.22155	1.1151 
9  	30    	1.20101	1.1151 
10 	21    	1.18003	1.1151 
11 	22    	1.16148	1.00992
12 	30    	1.12793	1.1081 
13 	37    	1.13192	1.1081 
14 	31    	1.122  	1.1081 
15 	29    	1.11005	1.09851
16 	33    	1.1409 	1.09851
17 	27    	1.13018	1.09811
18 	32    	1.12209	1.09811
19 	32    	1.10839	1.02488
20 	32    	1.12196	1.01031
21 	35    	1.10251	1.01031
22 	31    	1.06568	1.00992
23 	26    	1.03682	1.00992
24 	35    	1.02598	1.00992
25 	23    	1.04068	1.00992
26 	29    	1.02326	1.00992
27 	31    	1.03061	1.00992
28 	33    	1.03152	1.00992
29 	25    	1.01641	1.00992
30 	30    	1.02391	1.00992
Top 1 solution: [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0]
Fitness: 1.0099170039451253

------------wight1--------------
0.5

C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


gen	nevals	avg    	min    
0  	50    	1.42616	1.30343
1  	27    	1.3986 	1.24058
2  	31    	1.37935	1.28517
3  	37    	1.36363	1.26694
4  	34    	1.34981	1.26694
5  	31    	1.3255 	1.19006
6  	31    	1.28205	1.19006
7  	34    	1.25509	1.19006
8  	35    	1.22227	1.11549
9  	30    	1.20177	1.11549
10 	21    	1.18073	1.11549
11 	22    	1.16207	1.01016
12 	30    	1.12835	1.10845
13 	37    	1.1323 	1.10845
14 	31    	1.1224 	1.10845
15 	29    	1.11041	1.09883
16 	33    	1.14129	1.09883
17 	27    	1.13059	1.09826
18 	32    	1.12247	1.09826
19 	32    	1.10873	1.02509
20 	32    	1.12229	1.0105 
21 	35    	1.10277	1.0105 
22 	31    	1.06589	1.01016
23 	26    	1.03704	1.01016
24 	35    	1.0262 	1.01016
25 	23    	1.0409 	1.01016
26 	29    	1.02348	1.01016
27 	31    	1.03087	1.01016
28 	33    	1.03179	1.01016
29 	25    	1.01666	1.01016
30 	30    	1.02417	1.01016
Top 1 solution: [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0]
Fitness: 1.0101594281875497

------------wight1--------------
0.5

C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


gen	nevals	avg   	min    
0  	50    	1.4284	1.30362
1  	27    	1.40083	1.24133
2  	31    	1.38181	1.28565
3  	37    	1.36643	1.26753
4  	34    	1.35198	1.26753
5  	31    	1.32711	1.19082
6  	31    	1.2829 	1.19082
7  	34    	1.25577	1.19082
8  	35    	1.223  	1.11587
9  	30    	1.20253	1.11587
10 	21    	1.18143	1.11587
11 	22    	1.16266	1.0104 
12 	30    	1.12877	1.1088 
13 	37    	1.13269	1.1088 
14 	31    	1.1228 	1.1088 
15 	29    	1.11078	1.09915
16 	33    	1.14168	1.09915
17 	27    	1.131  	1.09842
18 	32    	1.12284	1.09842
19 	32    	1.10907	1.02531
20 	32    	1.12261	1.01069
21 	35    	1.10303	1.01069
22 	31    	1.06609	1.0104 
23 	26    	1.03727	1.0104 
24 	35    	1.02642	1.0104 
25 	23    	1.04112	1.0104 
26 	29    	1.02371	1.0104 
27 	31    	1.03112	1.0104 
28 	33    	1.03205	1.0104 
29 	25    	1.01691	1.0104 
30 	30    	1.02443	1.0104 
Top 1 solution: [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0]
Fitness: 1.0104018524299738

------------wight1--------------
0.5
-